Kaggle link: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import everything needed

In [ ]:
import zipfile
import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

wandb.login(key=secret_value_0)
wandb.init(project='CatvsDog-11', save_code=True) #, mode="disabled")

## Unzip datasets

In [ ]:
train_dir = 'train'
test_dir = 'test'
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip') as train_zip:
    train_zip.extractall('')
    
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip') as test_zip:
    test_zip.extractall('')
train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))
print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

In [ ]:
labels = [path.split('/')[-1].split('.')[0] for path in train_list]

## Use Sklearn to split data

In [ ]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=labels,
                                          random_state=0)
print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

We will discuss this in more detail in a near future...

In [ ]:

# 图像增强
dataset_transform = {
    'train': transforms.Compose([
        transforms.RandomRotation(45),  # 随机反转-45到45度之间
        transforms.Resize(300),
        transforms.CenterCrop(224),  # 从中心开始裁剪224
        transforms.RandomHorizontalFlip(p=0.5),  # 随机水平翻转， 选择一个概率
        transforms.RandomVerticalFlip(p=0.5),  # 随机垂直翻转
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),
        # 参数1为亮度， 参数2为对比度， 参数3为饱和度， 参数4为色相
        transforms.RandomGrayscale(p=0.025),  # 概率转换成灰度率， 3通道是R G B
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # 均值 标准差
    ]),

    'vaild': transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),

    'test': transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
}


Define the dataset using PIL to read image

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        self.filelength = len(file_list)

    def __len__(self):
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = img_path.split("/")[-1].split(".")[0]
        label = 1 if label == "dog" else 0
        return img_transformed, label

In [ ]:
train_data = CatsDogsDataset(train_list, transform=dataset_transform['train'])
valid_data = CatsDogsDataset(valid_list, transform=dataset_transform['vaild'])
test_data = CatsDogsDataset(test_list, transform=dataset_transform['test'])

Create dataloader, you can modify the batch size if needed

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
valid_loader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
!pip install torchvision --upgrade

In [ ]:
import torchvision
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Sequential(
           nn.Linear(in_features=512, out_features=2, bias=True)         
 )


net = model.cuda()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Detected device: {}'.format(device))
net.to(device)

num_epochs = 30
lr = 5e-3

criterion = nn.CrossEntropyLoss()
params = net.parameters()
# optimizer = torch.optim.AdamW(params, lr=lr,weight_decay=1e-3)
optimizer = torch.optim.SGD(params, lr=lr)

wandb.watch(net, log="all", criterion=criterion, log_freq=1,  log_graph=(True))
params

In [ ]:
for epoch in range(num_epochs):
    net.train()

    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        prediction = net(X)
        loss = criterion(prediction, y)
        loss.backward()
        optimizer.step()

        _, predictions = torch.max(prediction, 1)
        
        wandb.log({
            'mlp/train_loss': loss.item(),
            'mlp/train_accuracy': (y == predictions).sum()/len(y),
        })


    with torch.no_grad():
        net.eval()
        for X, y in valid_loader:
            X = X.to(device)
            y = y.to(device)

            prediction = net(X)

            loss = criterion(prediction, y)
            _, predictions = torch.max(prediction, 1)
        
            wandb.log({
                'mlp/val_loss': loss.item(),
                'mlp/val_accuracy': (y == predictions).sum()/len(y),
            })


# **Submission**

In [ ]:
with torch.no_grad():
    net.eval()
    test_pred = torch.LongTensor()
    test_pred = test_pred.to(device)
    for data, i in test_loader:
        data = data.to(device)

        prediction = net(data)
        dog_props =  F.softmax(prediction, dim=1)[:, 1]
        test_pred = torch.cat((test_pred, dog_props), dim=0)
    out_df = pd.DataFrame(np.c_[np.arange(1, len(test_list)+1)[:, None],
                                test_pred.cpu().numpy()], columns=['ImageId', 'Label'])
    # out_df = out_df.astype({'id':'str', 'label':'str'})
    out_df.to_csv('submission.csv', index=False)

out_df